# Tile creation from Open Topo dataset


In [2]:
!python --version

Python 3.10.6


In [3]:
! pip install --upgrade pip

In [4]:
!python -m pip install --upgrade pip

In [5]:
! pip install laspy

In [6]:
! python3 -m pip install "laspy[lazrs,laszip]"

In [7]:
! pip install numpy
! pip install scipy
! pip install rvt-py
! pip install GDAL

  Obtaining dependency information for rvt-py from https://files.pythonhosted.org/packages/40/b4/72d0cccd4a473b179c12dc92b3116ed6654728dfe2d9a8d318b921888a2c/rvt_py-2.2.1-py3-none-any.whl.metadata
  Using cached rvt_py-2.2.1-py3-none-any.whl.metadata (6.0 kB)
  Using cached GDAL-3.7.1.1.tar.gz (776 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      running egg_info
      creating /private/var/folders/fk/kzkbr5cn0v1fv4g2thf4pb5h0000gn/T/pip-pip-egg-info-1v6bsqgo/GDAL.egg-info
      writing /private/var/folders/fk/kzkbr5cn0v1fv4g2thf4pb5h0000gn/T/pip-pip-egg-info-1v6bsqgo/GDAL.egg-info/PKG-INFO
      writing dependency_links to /private/var/folders/fk/kzkbr5cn0v1fv4g2thf4pb5h0000gn/T/pip-pip-egg-info-1v6bsqgo/GDAL.egg-info/dependency_links.txt
      writing requirements to /private/var/folders/fk/kzkbr5cn0v1fv4g2thf4pb5h0000gn/T/pip-pip-egg-info-1v6

In [5]:
#! pip install "laspy[lazrs]"

! pip install rasterio
! pip install folium

In [33]:
import laspy
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rvt.default
import rvt.blend
import rasterio
import geopandas as gpd
import os
import pyproj

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

ModuleNotFoundError: No module named 'rvt'

In [21]:
#path to folder where point_cloud (lidar) tiles are and jpeg-tiles will be created.
path = "/Users/user/code/flegoualher/maia/raw_data/test_data/MX19_Inomata_complete"

In [20]:
#Move lax files to another folder

import os
import shutil


def move_lax_files(source_folder, destination_folder):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get a list of all files in the source folder
    file_list = os.listdir(source_folder)

    # Loop through the files and move .lax files to the destination folder
    for file_name in file_list:
        if file_name.endswith('.lax'):
            source_file = os.path.join(source_folder, file_name)
            destination_file = os.path.join(destination_folder, file_name)
            shutil.move(source_file, destination_file)
            print(f"Moved {file_name} to {destination_folder}")

if __name__ == "__main__":
    source_folder = "/Users/user/code/flegoualher/maia/raw_data/test_data/MX19_Inomata_complete"
    destination_folder = "/Users/user/code/flegoualher/maia/raw_data/test_data/lax_files"

    move_lax_files(source_folder, destination_folder)


Moved 78500_1960000.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 675000_1927500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 717500_1992500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 78500_1964500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 714000_1987000.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 701000_1987000.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 676500_1936500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 663500_1936500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 701000_1983500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 714000_1983500.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
Moved 710000_1993000.lax to /Users/user/code/flegoualher/maia/raw_data/test_data/lax_files
M

In [22]:
def display_elevation(df_elevation):
    dem_arr=df_elevation['z'].to_numpy().reshape(WIDTH+1,HEIGHT+1)
    default = rvt.default.DefaultValues()
    default.slp_output_units = "degree"
    slope_arr = default.get_slope(dem_arr=dem_arr, resolution_x=SAMPLING_IN_M, resolution_y=SAMPLING_IN_M)
#   plt.imshow(slope_arr, cmap='gray_r')
    slope_arr = np.clip(slope_arr/(55),0,1)
    #slope_arr = (slope_arr-np.amin(slope_arr))/(np.amax(slope_arr)-np.amin(slope_arr))
#   print(f'Values of slope are between {slope_arr.min()} and {slope_arr.max()}')
    return slope_arr


In [23]:
def compute_slope(df_elevation, width, height):
    dem_arr=df_elevation['z'].to_numpy().reshape(width,height)
    default = rvt.default.DefaultValues()
    default.slp_output_units = "degree"
    slope_arr = default.get_slope(dem_arr=dem_arr, resolution_x=SAMPLING_IN_M, resolution_y=SAMPLING_IN_M)
#   plt.imshow(slope_arr, cmap='gray_r')
    slope_arr = np.clip(slope_arr/(55),0,1)
    #slope_arr = (slope_arr-np.amin(slope_arr))/(np.amax(slope_arr)-np.amin(slope_arr))
#   print(f'Values of slope are between {slope_arr.min()} and {slope_arr.max()}')
    return slope_arr

In [24]:
def compute_svf_opns(df_elevation, width, height):
    dem_arr=df_elevation['z'].to_numpy().reshape(width,height)
    default = rvt.default.DefaultValues()
    svf_n_dir = 16  # number of directions
    svf_r_max = int(5/SAMPLING_IN_M) # max search radius in pixels
    svf_noise = 0  # level of noise remove (0-don't remove, 1-low, 2-med, 3-high)

    svf_opns_dict = default.get_sky_view_factor(dem_arr=dem_arr, resolution=SAMPLING_IN_M,
                                                    compute_svf=True, compute_asvf=False, compute_opns=True)

    svf_arr = svf_opns_dict["svf"]
    #svf_arr = np.clip((svf_arr-0.65)/0.35,0,1)
    svf_arr = (svf_arr-np.amin(svf_arr))/(np.amax(svf_arr)-np.amin(svf_arr))
    #plt.imshow(svf_arr, cmap='gray')

    #print(f'Skyview factor values are between {svf_arr.min()} and {svf_arr.max()}')

    opns_arr = svf_opns_dict["opns"]
    opns_arr = np.clip((opns_arr-60)/35,0,1)
    #opns_arr = (opns_arr-np.amin(opns_arr))/(np.amax(opns_arr)-np.amin(opns_arr))
    #plt.imshow(opns_arr, cmap='gray')

    #print(f'Positive openness are between {opns_arr.min()} and {opns_arr.max()}')

    return svf_arr, opns_arr

In [25]:
def display_VAT_HS(df_elevation,width,height):
    slope_arr = 1 - compute_slope(df_elevation,width,height)
    svf_arr, opns_arr = compute_svf_opns(df_elevation,width,height)
    VAT_HS = np.dstack((slope_arr,opns_arr, svf_arr))
    return VAT_HS

In [26]:
#coords_input[:][0]


In [30]:
def create_tile(tile_name):
    #Load tile and create df
    las_file = laspy.read(f"{path}/{tile_name}.laz")
    points = np.vstack((las_file.x, las_file.y, las_file.z, las_file.classification)).transpose()
    df_elevation_input = pd.DataFrame(points, columns = ['x','y','z','category'])
    df_elevation_input = df_elevation_input[df_elevation_input['category']==2.0]
    df_elevation_input.drop(columns=['category'])
    print(f"Create tile {tile_name}'s jpeg image")
    #print(df_elevation_input.shape)

    #Create elevation input
    #proj = "EPSG:32615"

    # Get X and Y coordinates of rainfall points
    x_coordinates_input = df_elevation_input.x
    y_coordinates_input = df_elevation_input.y

    #Create list of XY coordinate pairs
    coords_input = [list(xy) for xy in zip(x_coordinates_input, y_coordinates_input)]
    elevation_input = list(df_elevation_input.z)

    # Split data into testing and training sets
    coords_train, coords_test, elevation_train, elevation_test = train_test_split(coords_input, elevation_input, test_size = 0.20)

    # Set number of neighbors to look for
    neighbors = 100

    # Initialize KNN regressor
    knn_regressor = KNeighborsRegressor(n_neighbors = neighbors, weights = "distance")

    # Fit regressor to data
    knn_regressor.fit(coords_train, elevation_train)

    # Defining regular grid
    SAMPLING_IN_M = 1

    X_MIN, X_MAX = min(df_elevation_input.x), max(df_elevation_input.x)#
    # print('X range in m: {} - {}'.format(round(X_MIN, 0),round(X_MAX, 0)))

    Y_MIN, Y_MAX = min(df_elevation_input.y), max(df_elevation_input.y)
    # print('Y range in m: {} - {}'.format(round(Y_MIN, 0),round(Y_MAX, 0)))

    x_coordinates_output = np.arange(X_MIN,X_MAX+SAMPLING_IN_M,SAMPLING_IN_M)
    y_coordinates_output = np.arange(Y_MIN,Y_MAX+SAMPLING_IN_M,SAMPLING_IN_M)

    width = len(x_coordinates_output)
    # print('WIDTH :', width)
    height = len(y_coordinates_output)
    # print('HEIGHT :', height)

    # print('Output image dimensions: Width: {} samples - Height: {} samples '.format(width,height))
    # print('X,Y coordinates dimensions : x: {}, y: {} '.format(len(x_coordinates_output),len(y_coordinates_output)))

    coords_output = []
    for x in x_coordinates_output:
        for y in y_coordinates_output:
            coords_output.append([x,y])


    df_elevation_output = pd.DataFrame(coords_output, columns =['x', 'y'])
    # print('Elevation output x,y size', df_elevation_output.shape)
    df_elevation_output['z'] = knn_regressor.predict(coords_output)
    # print('Prediction output z size', len(df_elevation_output['z']))

    # Filter out if point too far away
    MAX_DISTANCE = 100

    for coord in coords_output:
      distances, indices = knn_regressor.kneighbors(coord)

    #compute layers
    slope_arr = compute_slope(df_elevation_output, width, height)
    svf_arr, opns_arr = compute_svf_opns(df_elevation_output, width, height)

    #create rgb_image
    rgb_image = display_VAT_HS(df_elevation_output,width,height)
    plt.imshow(rgb_image)
    plt.axis('off')
    plt.show()

    # Convert the VAT_HS array to PIL Image
    image = Image.fromarray(np.uint8(rgb_image * 255))

    # Save the image as a JPEG file
    print(f"Tile {tile_name}.jpeg created ! ✅ ")
    return image.save(f"{path}/jpeg_tiles/{tile_name}.jpeg")

In [31]:
coords_output[0]

NameError: name 'coords_output' is not defined

In [32]:
create_tile('61000_1951000')

Create tile 61000_1951000's jpeg image


NameError: name 'train_test_split' is not defined

In [ ]:
create_tile('61000_1951500')

In [ ]:
create_tile('61500_1952000')

In [ ]:
# List all files in the folder
files = os.listdir(f'{path}/point_cloud_tiles/')
files = [file_name[:-4] if file_name.endswith(".laz") else file_name for file_name in files]
files

In [ ]:
for elt in files:
  create_tile(elt)

#GPS coordinates

In [ ]:
#Transform X and Y into longitude and latitude

# Define the source and target coordinate systems
source_crs = pyproj.CRS.from_epsg(32615)  # WGS 1984 UTM Zone 15N
target_crs = pyproj.CRS.from_epsg(4326)   # WGS84 latitude/longitude

# Create a transformation object
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Example coordinate transformation
x = 661499.94  # Replace with your UTM x-coordinate
y = 1952370.92  # Replace with your UTM y-coordinate
lon, lat = transformer.transform(x, y)

# Output the transformed longitude and latitude
print(f"Longitude: {lon}")
print(f"Latitude: {lat}")

In [ ]:
#Load tile and create df and calculate longitude and latitude
tile_name = '61000_1951000'
las_file = laspy.read(f"{path}/point_cloud_tiles/{tile_name}.laz")
points = np.vstack((las_file.x, las_file.y, las_file.z)).transpose()
df_points = pd.DataFrame(points, columns = ['x','y','z'])
df_points['longitude'], df_points['latitude'] = transformer.transform(df_points['x'], df_points['y'])
df_points.head(20)


In [ ]:
# Create a folium map centered on the specified coordinates lat, lon
map = folium.Map(location=[lat, lon], zoom_start=10)
folium.Marker([latitude, longitude]).add_to(map)
map
